# Toronto Neighborhood classification
We will explore, segment, and cluster the neighborhoods in the city of Toronto. It's a part of IBM  
Applied Data Science Capstone. 
For the Toronto neighborhood data, a Wikipedia page exists that has all the information we need to explore and cluster the neighborhoods in Toronto. We will be scrape the Wikipedia page and wrangle the data, clean it, and then read it into a pandas dataframe.
Once the data is in a structured format, we can applly required analysis to explore and cluster the neighborhoods in the city of Toronto.
In this notebook we will just create the dataframe.

First we will import the required libraries .

In [253]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

import folium # plotting library
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
from pandas.io.html import read_html
import requests
from bs4 import BeautifulSoup


print('Libraries imported.')

Libraries imported.


The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood.
WE will use the BeautifulSoup package to transform the data in the table on the Wikipedia page into the above pandas dataframe.

In [ ]:


page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'


res = requests.get(page)
soup = BeautifulSoup(res.content,'lxml')
table = soup.find_all('table')[0] 
dfs = pd.read_html(str(table))

df = dfs[0]
df.columns = df.iloc[0]
df.drop([df.index[0]], inplace=True,axis=0, errors='ignore')

df

Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [ ]:
df=df.dropna()
df = df.drop(df[df['Borough']=="Not assigned"].index)


If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.


In [ ]:
df.loc[df['Neighbourhood']=="Not assigned","Neighbourhood"]=df.loc[df['Neighbourhood']=="Not assigned","Borough"]

More than one neighborhood can exist in one postal code area. These rows will be combined into one row with the neighborhoods separated with a comma.

In [ ]:
df_grouped = df.groupby(['Postcode','Borough'])['Neighbourhood'].apply(','.join).reset_index()
df_grouped

print the number of rows of your dataframe.

In [252]:
df.shape

(212, 3)

Now that we have built a dataframe of the postal code of each neighborhood along with the borough name and neighborhood name, in order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood.

In [255]:
df_postal = pd.read_csv("https://cocl.us/Geospatial_data")
df_postal.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Then we will merge the two dataframes to get a dataframe of [Postcode , Borough , Neighbourhood , Latitude, Longitude]

In [259]:
df_merged = pd.merge(df_grouped, df_postal, how='left', left_on=['Postcode'], right_on=['Postal Code'])

In [260]:
df_merged.drop(columns=['Postal Code'], inplace= True)
df_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
